In [1]:
import sys
sys.path.append("..")

In [2]:
from tqdm import tqdm

import torch
import torch.optim as optim
import torch.nn as nn

from vision_transformer.dataset import iterator
from torch.utils.data import DataLoader
from vision_transformer.layers import classifier

In [3]:
train_iterator = iterator.ImageNetIterator(is_train=True)
valid_iterator = iterator.ImageNetIterator(is_train=False)

train_loader = DataLoader(train_iterator, batch_size=32*2, shuffle=True, num_workers=10)
valid_loader = DataLoader(valid_iterator, batch_size=32*2, shuffle=False, num_workers=10)

___

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import torchvision.models as models
model = models.resnet18(True)
model.fc = nn.Linear(512, len(train_iterator.label_dict)) # add new lasy fc layer
model = nn.DataParallel(model).to(device) # paralleize

In [6]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [7]:
def train() : 
    model.train()
    losses = []
    accuracies = []

    for x,y in tqdm(train_loader, desc='train') : 
        pred = model(x.to(device))

        optimizer.zero_grad()
        loss = criterion(pred, y.to(device))
        loss.backward()
        optimizer.step()

        correct = (torch.argmax(pred, dim=1) == y.to(device)).sum()
        acc = correct.item() / y.shape[0]

        accuracies.append(acc)
        losses.append(loss.item())

    agg_acc = sum(accuracies) / len(accuracies)
    agg_loss = sum(losses) / len(losses)
    return agg_acc, agg_loss

def evalulate() : 
    model.eval()
    losses = []
    accuracies = []

    for x,y in tqdm(valid_loader, desc='valid') : 
        pred = model(x.to(device))

        loss = criterion(pred, y.to(device))
        correct = (torch.argmax(pred, dim=1) == y.to(device)).sum()
        acc = correct.item() / y.shape[0]

        accuracies.append(acc)
        losses.append(loss.item())

    agg_acc = sum(accuracies) / len(accuracies)
    agg_loss = sum(losses) / len(losses)
    return agg_acc, agg_loss

In [8]:
epoches = 20

for proc in range(epoches) : 
    t_acc, t_loss = train()
    v_acc, v_loss = evalulate()
    print(f"""
                === {proc+1}th Epoch ===
    
        Train Loss : {round(t_loss, 3)} | Train Acc : {round(t_acc, 3)}
        Valid Loss : {round(v_loss, 3)} | Valid Acc : {round(v_acc, 3)}
        
        ============================================
        ============================================
    """)

valid: 100%|██████████| 516/516 [00:36<00:00, 14.12it/s]



                === 1th Epoch ===
    
        Train Loss : 1.83 | Train Acc : 0.515
        Valid Loss : 1.6 | Valid Acc : 0.574
        
    


valid: 100%|██████████| 516/516 [00:36<00:00, 14.02it/s]



                === 2th Epoch ===
    
        Train Loss : 1.266 | Train Acc : 0.649
        Valid Loss : 1.413 | Valid Acc : 0.617
        
    


valid: 100%|██████████| 516/516 [00:36<00:00, 14.04it/s]


...


                === 18th Epoch ===
    
        Train Loss : 0.244 | Train Acc : 0.924
        Valid Loss : 1.471 | Valid Acc : 0.694
        
    


valid: 100%|██████████| 516/516 [00:37<00:00, 13.92it/s]



                === 19th Epoch ===
    
        Train Loss : 0.236 | Train Acc : 0.925
        Valid Loss : 1.398 | Valid Acc : 0.712
        
    


valid: 100%|██████████| 516/516 [00:36<00:00, 14.13it/s]


                === 20th Epoch ===
    
        Train Loss : 0.214 | Train Acc : 0.932
        Valid Loss : 1.465 | Valid Acc : 0.71
        
    


In [10]:
torch.save(model.state_dict(), 'resnet18_imagenet_pretrained.pt')